In [1]:
import csv
import requests
import pandas as pd
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
emergency_generation_csv_path = "emergency_generation.csv"
demand_csv_path = "demand.csv"
breakdowns_csv_path = "breakdowns.csv"

emergency_generation_csv = "https://www.eskom.co.za/dataportal/wp-content/uploads/2022/03/Pumped_storage_gen_hours_gas_generation_and_manual_load_reduction.csv"
breakdowns_csv = "https://www.eskom.co.za/dataportal/wp-content/uploads/2022/03/Hourly_UCLF_and_OCLF_Trend.csv"
demand_csv = "https://www.eskom.co.za/dataportal/wp-content/uploads/2022/03/System_hourly_demand_and_available_capacity.csv"


emergency_generation = pd.read_csv(emergency_generation_csv)
breakdowns = pd.read_csv(breakdowns_csv)
demand = pd.read_csv(demand_csv)

# print(emergency_generation)
# print(breakdowns)


In [3]:
temp = pd.read_csv(emergency_generation_csv_path)
emergency_generation = pd.concat((emergency_generation, temp), axis=0)
emergency_generation = emergency_generation.where(emergency_generation['Drakensberg Gen Unit Hours']>0)
emergency_generation = emergency_generation.drop_duplicates(subset=['Date'])
emergency_generation = emergency_generation.sort_values(['Date'])
emergency_generation['Date'] = pd.to_datetime(emergency_generation['Date'])

temp = pd.read_csv(breakdowns_csv_path)
breakdowns = pd.concat((breakdowns, temp), axis=0)
breakdowns = breakdowns.drop_duplicates(subset=['DateTimeKey']).sort_values(['DateTimeKey'])
breakdowns['DateTimeKey'] = pd.to_datetime(breakdowns['DateTimeKey'])


temp = pd.read_csv(demand_csv_path)
demand = pd.concat((demand, temp), axis=0)
demand = demand.where(demand['Available Capacity Incl Non Comm Units'].notnull()).where(demand['Available Capacity Incl Non Comm Units']>0)
demand = demand.drop_duplicates(subset=['DateTimeKey']).sort_values(['DateTimeKey'])
demand['DateTimeKey'] = pd.to_datetime(demand['DateTimeKey'])



In [4]:

emergency_generation.to_csv(emergency_generation_csv_path)

demand.to_csv(demand_csv_path)

breakdowns.to_csv(breakdowns_csv_path)

In [5]:
fig = px.line(breakdowns, x = 'DateTimeKey', y = 'Hourly UCLF+OCLF')
fig.show()

In [6]:
demand["Surplus"] = demand['Available Capacity Incl Non Comm Units'] - demand['RSA Contracted Demand']

surplusFig = px.line(demand, x = 'DateTimeKey', y='Surplus')
surplusFig.show()

In [7]:
demandfig = px.line(demand, x = 'DateTimeKey', y = ['RSA Contracted Demand','Available Capacity Incl Non Comm Units'])
demandfig.show()

In [8]:
shedding = pd.merge(breakdowns, emergency_generation, how='left', left_on='DateTimeKey', right_on='Date')

sheddingfig2 = make_subplots(specs=[[{"secondary_y": True}]])
sheddingfig2.add_trace(
    go.Scatter(x=shedding['DateTimeKey'], y = shedding['Hourly UCLF+OCLF'], name='Outages'),
    secondary_y=False,
)
sheddingfig2.add_trace(
    go.Bar(x=shedding['DateTimeKey'], y = shedding['Manual Load Reduction(MLR)']/1000, name='Loadshedding Stage'),
    secondary_y=True,
)
sheddingfig2.show()

In [11]:
maxDemand = demand.sort_values('RSA Contracted Demand').groupby(demand.DateTimeKey.dt.to_period('D')).last().reset_index(drop=True)

maxDemand['MinSurplus'] = (maxDemand['Available Capacity Incl Non Comm Units']/1000) - (maxDemand['RSA Contracted Demand']/1000)
maxDemand['DateTimeKey'] = pd.to_datetime(maxDemand['DateTimeKey'])

# countdown = px.line(maxDemand.where(maxDemand['DateTimeKey'].dt.weekday <5), x='DateTimeKey', y='MinSurplus')
countdown = px.line(maxDemand, x='DateTimeKey', y='MinSurplus')
countdown.show()

